In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')


In [ ]:
amazon_data = pd.read_csv('amazon_stock_price.csv')


In [ ]:
amazon_data


In [ ]:
amazon_data.head()

In [ ]:
amazon_data.tail()

In [ ]:
amazon_data.shape

In [ ]:
amazon_data.info()

In [ ]:
amazon_data.describe()

In [ ]:
amazon_data.columns

In [ ]:
amazon_data.isnull().sum()

In [ ]:
amazon_data = amazon_data.drop(columns=['Adj Close'])


In [ ]:
amazon_data.head()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 8))

ax.plot(amazon_data['Date'],
        amazon_data['Close'],
        color='Red')

ax.xaxis.set_major_locator(plt.MaxNLocator(15))
ax.set_xlabel('Date', fontsize='11')

ax.set_ylabel('Price in USD', fontsize='11')

plt.title('Amazon Stock Prices')
plt.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 8))

ax.bar(amazon_data['Date'],
       amazon_data['Volume'])

ax.xaxis.set_major_locator(plt.MaxNLocator(15))

ax.set_xlabel('Date', fontsize='11')
ax.set_ylabel('Volumes', fontsize='11')
plt.title('Volume Trends', fontsize='20')

plt.grid()
plt.show()

In [ ]:
amazon_data['Market Cap'] = amazon_data['Open'] * amazon_data['Volume']

In [ ]:
fig, ax = plt.subplots(figsize=(20, 8))

ax.plot(amazon_data['Date'], amazon_data['Market Cap'], color='orange')

ax.xaxis.set_major_locator(plt.MaxNLocator(15))
ax.set_xlabel('Date', fontsize='11')
ax.set_ylabel('Market Cap', fontsize='11')

plt.title('Market Cap')
plt.grid()
plt.show()

In [ ]:
amazon_data.iloc[amazon_data['Market Cap'].    argmax()]

In [ ]:
ohlc = amazon_data[(amazon_data['Date'] > '1997-05-15')]
ohlc = ohlc.loc[:, ['Date',
                    'Open',
                    'High',
                    'Low',
                    'Close',
                    'Volume',
                    'Market Cap'
                    ]]
ohlc['Date'] = pd.to_datetime(ohlc['Date'], format='%d-%m-%Y')

fig, ax = plt.subplots(figsize=(20, 8))

ax.plot(ohlc['Date'], ohlc['Market Cap'], color='orange')
ax.xaxis.set_major_locator(plt.MaxNLocator(5))

ax.set_xlabel('Date', fontsize='11')
ax.set_ylabel('Market Cap', fontsize='11')

plt.grid()
plt.title('Market Cap (After 15 May, 1997)', fontsize='20')
plt.show()

In [ ]:
amazon_data['vol'] = (amazon_data['Close'] /
                      amazon_data['Close'].    shift(1))-1

In [ ]:
fig, ax = plt.subplots(figsize=(20, 8))
ax.plot(amazon_data['Date'], amazon_data['vol'], color='purple')

ax.xaxis.set_major_locator(plt.MaxNLocator(15))

plt.title('Volatility')

plt.grid()
plt.show()

In [ ]:
amazon_data['vol'].hist(bins=100, color='orange')

In [ ]:
amazon_data['Cumulative Return'] = (1 + amazon_data['vol']).cumprod()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 8))

ax.plot(amazon_data['Date'],
        amazon_data['Cumulative Return'],    color='green')

ax.xaxis.set_major_locator(plt.MaxNLocator(15))
ax.set_xlabel('Date', fontsize='11')
ax.set_ylabel('Cumulative Return', fontsize='11')

plt.title('Cumulative Return')
plt.grid()
plt.show()

In [ ]:
ohlc = amazon_data[(amazon_data['Date'] > '1997-05-15')]
ohlc = ohlc.loc[:, ['Date',
                    'Open',
                    'High',
                    'Low',
                    'Close',
                    'Volume',
                    'Market Cap',
                    'Cum']]
ohlc['Date'] = pd.to_datetime(ohlc['Date'], format='%d-%m-%Y')

fig, ax = plt.subplots(figsize=(20, 8))

ax.plot(ohlc['Date'], ohlc['Cumulative Return'], color='green')
ax.xaxis.set_major_locator(plt.MaxNLocator(5))
ax.set_xlabel('Date', fontsize='11')
ax.set_ylabel('Cumulative Return', fontsize='11')

plt.grid()
plt.title('Cumulative Return (After 05 May, 1997)', fontsize='20')
plt.show()

In [ ]:
amazon_data.iloc[amazon_data['Cumulative Return'].    argmax()]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import math

In [ ]:
amazon_data['Date'] =
pd.to_datetime(amazon_data['Date'])
amazon_data.set_index('Date',   inplace=True)


In [ ]:
data = amazon_data.filter(['Close'])
dataset = data.values
training_data_len = math.ceil(len(dataset) * .8)

training_data_len

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)
scaled_data

In [ ]:
train_data = scaled_data[0:    training_data_len, :]

x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i - 60: i, 0])
    y_train.append(train_data[i, 0])
    if i <= 60:
        print(x_train)
        print(y_train)
print()

In [ ]:
x_train,y_train=np.array(    x_train),np.array(    y_train)

In [ ]:
x_train = np.reshape(x_train, 
                     (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

In [ ]:
model = Sequential()

model.add(LSTM(64, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(32))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error')

In [ ]:
model.fit(x_train,
          y_train, 
          batch_size=1,
          epochs=10)

In [ ]:
test_data = scaled_data[training_data_len - 60:, :]
x_test = []
y_test = dataset[training_data_len:, :]

for i in range(60, len(test_data)):
    x_test.append(test_data[i - 60: i, 0])

In [ ]:
x_test=np.array(x_test)

In [ ]:
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
x_test.shape

In [ ]:
predictions=model.predict(x_test)
predictions=scaler.inverse_transform(predictions)

In [ ]:
rmse = np.sqrt(np.mean(predictions - y_test) ** 2)
rmse

In [ ]:
train = data[: training_data_len]

valid = data[training_data_len:]
valid['Predictions'] = predictions

plt.figure(figsize=(16, 8))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price', fontsize=18)
plt.plot(train['Close'], linewidth=3.5)
plt.plot(valid[['Close', 'Predictions']], linewidth=3.5)
plt.legend(['Train', 'Valid', 'Predictions'], loc='upper_center')

In [ ]:
valid

In [ ]:
amazon_quote = pd.read_csv('amazon_stock_price.csv')
new_amazon_data = amazon_quote.filter(['Close'])
last_60_days = new_amazon_data[- 60:].values

last_60_days_scaled = scaler.transform(last_60_days)
X_test = []
X_test.append(last_60_days_scaled)
X_test = np.array(X_test)

X_test = np.reshape(
    X_test, (X_test.shape[0], X_test.shape[1],        1))


pred_price = model.predict(X_test)
pred_price = scaler.inverse_transform(pred_price)
pred_price